**Form4 filings Part 1**

In the following code cell, provide the code for your function.

In [16]:
# Code cell for part 1 - do not delete or modify this line
def CreateDataFrame(basefolder):
    import pandas as pd
    import numpy as np
    import os
    import re
    import xml.etree.ElementTree as ET

    #initialize subfolders and data
    path1=basefolder+"\\4"
    L=list(os.listdir(path1))
    k=0
    data=[]
    
    for Folder in L:
        k+=1
        if k % 500 == 0:
            print(k)
            
        #enter folder, extract XML file
        path=path1+"\\"+Folder
        files=list(os.listdir(path))
        sub_text = open(path+"\\"+"full-submission.txt")
        text=sub_text.read()
        pattern="\<ownershipDocument\>(\n|.)*\<\/ownershipDocument\>"
        m=re.search(pattern,text)
        if m:
            XML_file=text[m.start():m.end()]
        else: 
            continue
        root = ET.fromstring(XML_file)
        tags = [e.tag for e in root.iter()]
        
        #collect data from "good" files
        if 'rptOwnerName' in tags and 'nonDerivativeTransaction' in tags:
            #get ownership info
            it=root.findall("./*/*/rptOwnerName")
            for x in it:
                OwnerName=x.text
                
            it=root.findall("./*/reportingOwnerRelationship/isDirector")
            for x in it:
                isDir=x.text
                
            it=root.findall("./*/reportingOwnerRelationship/isOfficer")
            for x in it:
                isOff=x.text
                
            it=root.findall("./*/reportingOwnerRelationship/isTenPercentOwner")
            for x in it:
                isTen=x.text

            #transaction info
            for e in root.iter("nonDerivativeTransaction"):
                for x in e.iter("securityTitle"):
                    for t in x:
                        if t.tag=='value':
                            SecTitle=t.text

                for x in e.findall("./transactionDate/"):
                    if x.tag=='value':
                        TransDate=x.text

                for x in e.findall("./*/transactionShares/"):
                    if x.tag=='value':
                        Shares=x.text

                for x in e.findall("./*/transactionPricePerShare/"):
                    if x.tag=='value':
                        PPS=x.text

                for x in e.findall("./*/transactionAcquiredDisposedCode/"):
                    if x.tag=='value':
                        ADCode=x.text

                for x in e.findall("./postTransactionAmounts/sharesOwnedFollowingTransaction/"):
                    if x.tag=='value':
                        SharesAfter=x.text

                for x in e.findall("./ownershipNature/directOrIndirectOwnership/"):
                    if x.tag=='value':
                        DIOwner=x.text

                indiv_data=[Folder, OwnerName, isDir, isOff, isTen, SecTitle,TransDate,Shares,PPS,ADCode,SharesAfter,DIOwner]
                data.append(indiv_data)

    df=pd.DataFrame(data,columns=['Folder', 'OwnerName', 'isDir', 'isOff', 'isTen', 
                      'SecTitle','TransDate','Shares','PPS','ADCode','SharesAfter','DIOwner'])
    df.to_csv("final_data.csv",index=False) 
    return df
    

In [20]:
df=CreateDataFrame('C:\\Users\\Aaron Zoll\\Desktop\\Grad School\\Semester 3\\688\\Assignments\\final\\PFE')

500
1000
1500
2000
2500
3000
3500
4000
4500
